In [14]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from time import time
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA

In [15]:
"""To display up to 50 columns of dataset"""
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 8)

"""To display all outputs of each cell"""
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


np.random.seed(100)

"""To get a timing of each function decorated"""
def timing(f):
    """
    Decorator for timing functions
    Usage:
    @timing
    def function(&):
        pass
    """
    
    
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('function:%r took: %2.2f sec' % (f.__name__, end - start))
        return result
    return wrapper

'To display up to 50 columns of dataset'

'To display all outputs of each cell'

'To get a timing of each function decorated'

In [3]:
class Trainer():
    """Base processor to be used for all preparation"""""
    def __init__(self, input_directory, df_): #input_directory = path to the folder containing df. 
        self.input_directory = input_directory
        self.df = df_
                
    @timing
    def read(self):
        """Read raw data"""
        self.df = pd.read_csv(self.input_directory + self.df)
             
    @timing
    def preprocessing(self):
        """processing of the input data"""
            
        #type change to category where needed
        self.category_columns = ["Store", "StoreType", "Assortment", "DayOfWeek", "PromoInterval", "StateHoliday",
                                 "OpeningType", "Month", "WeekOfYear", "DateOfMonth", "AffectedByCompetition", 
                                 "AffectedByPromo2", "StateHoliday", "SchoolHoliday", "Promo", "Promo2"]
        
        for column in self.category_columns:
            self.df[column] = self.df[column].astype('category')
            
        self.binary_columns = ["AffectedByCompetition", "AffectedByPromo2", "SchoolHoliday", "Promo", "Promo2"]
     
        for column in self.binary_columns:
            self.df[column] = self.df[column].astype('bool')
            
        self.df = pd.get_dummies(self.df, columns=["StoreType", "Assortment", "DayOfWeek", "PromoInterval", 
                                                   "StateHoliday", "OpeningType", "Month", "WeekOfYear", 
                                                   "DateOfMonth", "StateHoliday"])
        
        #Scale Interval Variables
        #self.normalize = MinMaxScaler(feature_range = (0,1))
        
        
        #extract only data of open stores
        self.df_open = self.df[self.df["Open"] == 1]
        self.df = self.df_open
        
        #Re_indexation 
        self.df = self.df.reset_index(drop='True')
        
        """put the date as index"""
        self.df["Date"] = pd.to_datetime(self.df["Date"].astype(str), format = '%Y/%m/%d')
        self.df = self.df.set_index("Date")
            
            
        self.df = self.df.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", 
                                "Promo2SinceWeek", "Promo2SinceYear", "Promo2SinceDate", "CompetitionOpenSinceDate",
                               "Customers", "Open"], axis = 1)
        
        
        self.train_df = self.df["2013-01-01": "2015-06-13"]
        self.test_df = self.df["2015-06-14":]
             
        self.y_train, self.y_test = self.train_df[['Sales']], self.test_df[["Sales"]]
        #self.standardize = StandardScaler()
        #self.normalize = MinMaxScaler(feature_range = (0,1))
        
        #self.y_train = np.sqrt(self.y_train)
        #self.y_train = self.standardize.fit_transform(self.y_train)
        
        self.X_train = self.train_df.drop(["Sales"], axis = 1)
        self.X_test = self.test_df.drop(["Sales"], axis = 1)
        
        self.X_train_random, self.X_test_random, self.y_train_random, self.y_test_random = train_test_split(self.df.drop(["Sales"], axis = 1), self.df[['Sales']], test_size=0.2)
        
    
    @timing
    def model_fit(self):
        self.model = LinearRegression()
        self.model.fit(self.X_train, self.y_train)
    
    @timing
    def model_fit_random(self):
        self.model_random = LinearRegression()
        self.model_random.fit(self.X_train_random, self.y_train_random)     
        
    @timing
    def validation(self, nb_store):
        self.predicted = self.model.predict(self.X_test[self.X_test["Store"] == nb_store])
        self.predicted = self.predicted.reshape(-1, 1)
        #self.predicted = self.standardize.inverse_transform(self.predicted)
        #self.predicted = np.square(self.predicted)
        
        
        plt.figure(figsize=(18, 7), dpi= 80, facecolor='w', edgecolor='k')
        plt.plot(self.y_test[self.X_test["Store"] == nb_store].index, self.predicted, color = 'blue',label = 'Predicted values')
        plt.plot(self.y_test[self.X_test["Store"] == nb_store], color = 'red', label = 'Real values')
        plt.legend()
        
        self.errors = abs(self.predicted - self.y_test[self.X_test["Store"] == nb_store])
        
        print('Mean Absolute Error:', round(np.mean(self.errors), 2))
        print('RMSE', round(m.sqrt(mean_squared_error(self.predicted, self.y_test[self.X_test["Store"] == nb_store])), 2))
        return 
    
    @timing
    def rmse(self):
        self.predicted = self.model.predict(self.X_test)
        self.predicted = self.predicted.reshape(-1, 1)
        #self.predicted = self.standardize.inverse_transform(self.predicted)
        #self.predicted = np.square(self.predicted)

        self.errors = abs(self.predicted - self.y_test)
        print('Mean Absolute Error:', round(np.mean(self.errors), 2))
        print('RMSE', round(m.sqrt(mean_squared_error(self.predicted, self.y_test)), 2))
        
    @timing
    def rmse_random(self):
        self.predicted_ran = self.model_random.predict(self.X_test_random)
        self.predicted_ran = self.predicted_ran.reshape(-1, 1)
        #self.predicted_ran = self.standardize.inverse_transform(self.predicted_ran)
        #self.predicted_ran = np.square(self.predicted_ran)

        self.errors_ran = abs(self.predicted_ran - self.y_test_random)
        print('Mean Absolute Error:', round(np.mean(self.errors_ran), 2))
        print('RMSE', round(m.sqrt(mean_squared_error(self.predicted_ran, self.y_test_random)), 2))
        
    def save_result(self, nb_store):
        self.result = self.y_test[self.X_test["Store"] == nb_store]
        self.predicted_store = self.model.predict(self.X_test[self.X_test["Store"] == nb_store])
        self.error_store = abs(self.predicted_store.reshape(-1, 1) - self.y_test[self.X_test["Store"] == nb_store])

        self.result["predicted_LR"] = self.predicted_store
        self.result["error_LR"] = self.error_store
        
        return self.result
        

In [4]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()
project.preprocessing()

function:'read' took: 2.29 sec
function:'preprocessing' took: 4.55 sec


In [5]:
project.model_fit()

function:'model_fit' took: 18.98 sec


In [6]:
project.rmse()

Mean Absolute Error: Sales    1893.75
dtype: float64
RMSE 2616.08
function:'rmse' took: 0.98 sec


In [7]:
result_733 = project.save_result(733)

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
result_733 = result_733.reset_index(drop=False)
result_733.head()

,Date,Sales,predicted_LR,error_LR
0,2015-06-14,16452,12086.428707,4365.571293
1,2015-06-15,16710,14760.723335,1949.276665
2,2015-06-16,16007,13673.058087,2333.941913
3,2015-06-17,15036,13219.259204,1816.740796
4,2015-06-18,15269,13262.322016,2006.677984


In [9]:
result_733.to_csv("LinearReg_store733_6weeks.csv", index=False)

In [10]:
result_351 = project.save_result(351)
result_351 = result_351.reset_index(drop=False)
result_351.head()

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Sales,predicted_LR,error_LR
0,2015-06-15,9850,8271.548184,1578.451816
1,2015-06-16,7449,7183.882936,265.117064
2,2015-06-17,6829,6730.084053,98.915947
3,2015-06-18,6628,6773.146864,145.146864
4,2015-06-19,6716,6870.429171,154.429171


In [11]:
result_351.to_csv("LinearReg_store351_6weeks.csv", index=False)

In [12]:
result_279 = project.save_result(279)
result_279 = result_279.reset_index(drop=False)
result_279.head()

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Sales,predicted_LR,error_LR
0,2015-06-15,19655,9328.608126,10326.391874
1,2015-06-16,11692,8240.942877,3451.057123
2,2015-06-17,12551,7787.143994,4763.856006
3,2015-06-18,10696,7830.206806,2865.793194
4,2015-06-19,10873,7927.489113,2945.510887


In [13]:
result_279.to_csv("LinearReg_store279_6weeks.csv", index=False)

In [11]:
project.model_fit_random()

function:'model_fit_random' took: 11.18 sec


In [12]:
project.rmse_random()

Mean Absolute Error: Sales    1904.32
dtype: float64
RMSE 2615.46
function:'rmse_random' took: 1.67 sec


In [21]:
result_57 = project.save_result(57)
result_57 = result_57.reset_index(drop=False)
result_57.head()

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Sales,predicted_LR,error_LR
0,2015-06-15,15677,9950.172986,5726.827014
1,2015-06-16,11734,8862.507738,2871.492262
2,2015-06-17,9884,8408.708855,1475.291145
3,2015-06-18,10350,8451.771667,1898.228333
4,2015-06-19,9065,8549.053974,515.946026


In [ ]:
# try making feature selection

In [19]:
class Trainer_Store():
    """Base processor to be used for all preparation"""""
    def __init__(self, input_directory, df_): #input_directory = path to the folder containing df. 
        self.input_directory = input_directory
        self.df = df_
                
    @timing
    def read(self):
        """Read raw data"""
        self.df = pd.read_csv(self.input_directory + self.df)
             
    @timing
    def preprocessing(self):
        """processing of the input data"""
            
        #type change to category where needed
        self.category_columns = ["Store", "StoreType", "Assortment", "DayOfWeek", "PromoInterval", "StateHoliday",
                                 "OpeningType", "Month", "WeekOfYear", "DateOfMonth", "AffectedByCompetition", 
                                 "AffectedByPromo2", "StateHoliday", "SchoolHoliday", "Promo", "Promo2"]
        
        for column in self.category_columns:
            self.df[column] = self.df[column].astype('category')
            
        self.binary_columns = ["AffectedByCompetition", "AffectedByPromo2", "SchoolHoliday", "Promo", "Promo2"]
     
        for column in self.binary_columns:
            self.df[column] = self.df[column].astype('bool')
            
        self.df = pd.get_dummies(self.df, columns=["StoreType", "Assortment", "DayOfWeek", "PromoInterval", 
                                                   "StateHoliday", "OpeningType", "Month", "WeekOfYear", 
                                                   "DateOfMonth", "StateHoliday"])
        
        #Scale Interval Variables
        #self.normalize = MinMaxScaler(feature_range = (0,1))      
        
        #extract only data of open stores
        self.df_open = self.df[self.df["Open"] == 1]
        self.df = self.df_open
        
        #Re_indexation 
        self.df = self.df.reset_index(drop='True')
        
        """put the date as index"""
        self.df["Date"] = pd.to_datetime(self.df["Date"].astype(str), format = '%Y/%m/%d')
        self.df = self.df.set_index("Date")
                     
        self.df = self.df.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", 
                                "Promo2SinceWeek", "Promo2SinceYear", "Promo2SinceDate", "CompetitionOpenSinceDate",
                               "Customers", "Open"], axis = 1)

    
    @timing
    def model_fit(self):
        self.mae_list = []
        self.rmse_list = []
        for i in range(len(self.df["Store"].unique())):
            # Filter by Store
            self.subset = self.df[(self.df["Store"]==(i+1))]
            self.subset = self.subset.drop(["Store"], axis = 1)

            # Split train and test set  
            self.train_df = self.subset["2013-01-01": "2015-06-13"]
            self.test_df = self.subset["2015-06-14":]

            self.y_train, self.y_test = self.train_df[['Sales']], self.test_df[["Sales"]]

            self.X_train = self.train_df.drop(["Sales"], axis = 1)
            self.X_test = self.test_df.drop(["Sales"], axis = 1)

            self.model = LinearRegression()
            self.model.fit(self.X_train, self.y_train)
                        
            self.predicted = self.model.predict(self.X_test)

            self.errors = abs(self.predicted - self.y_test)
            self.mae = round(np.mean(self.errors), 2)
            self.mae_list.append(self.mae)
            
            self.predicted = self.predicted.reshape(-1, 1)
            self.rmse = round(m.sqrt(mean_squared_error(self.predicted, self.y_test)), 2)
            self.rmse_list.append(self.rmse)
            
        self.rmse_list = pd.DataFrame(self.rmse_list, columns=["RMSE"])
        self.mae_list = pd.DataFrame(self.mae_list, columns=["MAE"])

        self.store = list(range(1,1116))
        self.store_series = pd.DataFrame(self.store, columns=["Store"])
        self.result = pd.merge(self.store_series, self.rmse_list, how = 'left',left_index = True, right_index = True)
        self.result = pd.merge(self.result, self.mae_list, how = 'left',left_index = True, right_index = True)
 
        print('Mean Absolute Error:', self.result.MAE.mean())
        print('Median Absolute Error:', self.result.MAE.median())
        print('Mean of RMSE:', self.result.RMSE.mean())
        print('Median of RMSE:', self.result.RMSE.median())
        
    def save_summary(self):
        return self.result
        
    
    @timing
    def model_fit_random(self):
        self.X_train_random, self.X_test_random, self.y_train_random, self.y_test_random = train_test_split(self.df.drop(["Sales"], axis = 1), self.df[['Sales']], test_size=0.2)
        self.model_random = LinearRegression()
        self.model_random.fit(self.X_train_random, self.y_train_random)     
        
    @timing
    def validation(self, nb_store):
        self.predicted = self.model.predict(self.X_test[self.X_test["Store"] == nb_store])
        self.predicted = self.predicted.reshape(-1, 1)
        #self.predicted = self.standardize.inverse_transform(self.predicted)
        #self.predicted = np.square(self.predicted)
        
        
        plt.figure(figsize=(18, 7), dpi= 80, facecolor='w', edgecolor='k')
        plt.plot(self.y_test[self.X_test["Store"] == nb_store].index, self.predicted, color = 'blue',label = 'Predicted values')
        plt.plot(self.y_test[self.X_test["Store"] == nb_store], color = 'red', label = 'Real values')
        plt.legend()
        
        self.errors = abs(self.predicted - self.y_test[self.X_test["Store"] == nb_store])
        
        print('Mean Absolute Error:', round(np.mean(self.errors), 2))
        print('RMSE', round(m.sqrt(mean_squared_error(self.predicted, self.y_test[self.X_test["Store"] == nb_store])), 2))
        return 
    
    @timing
    def rmse(self):
        self.predicted = self.model.predict(self.X_test)
        self.predicted = self.predicted.reshape(-1, 1)
        #self.predicted = self.standardize.inverse_transform(self.predicted)
        #self.predicted = np.square(self.predicted)

        self.errors = abs(self.predicted - self.y_test)
        print('Mean Absolute Error:', round(np.mean(self.errors), 2))
        print('RMSE', round(m.sqrt(mean_squared_error(self.predicted, self.y_test)), 2))
        
    @timing
    def rmse_random(self):
        self.predicted_ran = self.model_random.predict(self.X_test_random)
        self.predicted_ran = self.predicted_ran.reshape(-1, 1)
        #self.predicted_ran = self.standardize.inverse_transform(self.predicted_ran)
        #self.predicted_ran = np.square(self.predicted_ran)

        self.errors_ran = abs(self.predicted_ran - self.y_test_random)
        print('Mean Absolute Error:', round(np.mean(self.errors_ran), 2))
        print('RMSE', round(m.sqrt(mean_squared_error(self.predicted_ran, self.y_test_random)), 2))
        
    def save_result(self, nb_store):
        self.result = self.y_test[self.X_test["Store"] == nb_store]
        self.predicted_store = self.model.predict(self.X_test[self.X_test["Store"] == nb_store])
        self.error_store = abs(self.predicted_store.reshape(-1, 1) - self.y_test[self.X_test["Store"] == nb_store])

        self.result["predicted_LR"] = self.predicted_store
        self.result["error_LR"] = self.error_store
        
        return self.result
        

In [20]:
new = Trainer_Store("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
new.read()
new.preprocessing()
new.model_fit()

result = new.save_summary()
result = result.reset_index(drop=False)
result.head()

function:'read' took: 2.25 sec
function:'preprocessing' took: 2.80 sec
Mean Absolute Error: nan
Median Absolute Error: nan
Mean of RMSE: 1.321318851401602e+16
Median of RMSE: 874.39
function:'model_fit' took: 24.96 sec


,index,Store,RMSE,MAE
0,0,1,424.28,NaN
1,1,2,695.28,NaN
2,2,3,1091.87,NaN
3,3,4,1390.49,NaN
4,4,5,746.53,NaN


In [18]:
result.to_csv("LinearReg_summary-result_6weeks.csv", index=False)